<a href="https://colab.research.google.com/github/khalidpark/Projects/blob/main/%EB%A0%88%EC%95%8C%EB%A7%88%EB%93%9C%EB%A6%AC%EB%93%9C%20%EC%8A%B9%EB%A6%AC%EC%98%88%EC%B8%A1/RealMadrid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pandas==0.20.0

In [ ]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing


In [ ]:
CSV_COLUMN_NAMES = ['MatchGap', 'Month', 'MatchType', 'HomeAway','Team', 'TeamRanking', 'PreScoreGoal', 'PrePreScoreGoal', 'PreGetGoal', 'PrePreGetGoal', 'RamosTime','RamosGoal', 'RamosAss','Results']
RESULTS = ['Win', 'Draw', 'Lose']

In [ ]:
train = pd.read_csv('/content/real_training2.csv', names=CSV_COLUMN_NAMES)

In [ ]:
train.head(15)

,MatchGap,Month,MatchType,HomeAway,Team,TeamRanking,PreScoreGoal,PrePreScoreGoal,PreGetGoal,PrePreGetGoal,RamosTime,RamosGoal,RamosAss,Results
0,5,9,1,1,60,12,0,2,3,3,90,0,0,2
1,4,9,1,0,8,7,0,0,1,3,68,0,0,0
2,3,9,1,1,25,20,3,0,1,1,0,0,0,0
3,3,9,0,0,53,0,3,3,0,1,90,0,0,0
4,4,10,1,0,61,19,2,3,1,0,90,0,0,0
5,13,10,1,1,9,10,4,2,0,1,68,0,0,0
6,4,10,0,0,69,0,3,4,0,0,90,0,0,0
7,4,10,1,0,87,5,4,3,1,0,0,0,0,2
8,3,10,1,1,26,9,1,4,2,1,90,0,0,2
9,3,10,1,1,62,14,1,1,3,2,90,0,0,0


In [ ]:
train_y = train.pop('Results')

# The label column has now been removed from the features.
train.head()


,MatchGap,Month,MatchType,HomeAway,Team,TeamRanking,PreScoreGoal,PrePreScoreGoal,PreGetGoal,PrePreGetGoal,RamosTime,RamosGoal,RamosAss
0,5,9,1,1,60,12,0,2,3,3,90,0,0
1,4,9,1,0,8,7,0,0,1,3,68,0,0
2,3,9,1,1,25,20,3,0,1,1,0,0,0
3,3,9,0,0,53,0,3,3,0,1,90,0,0
4,4,10,1,0,61,19,2,3,1,0,90,0,0


In [ ]:
def input_evaluation_set():
    features = {'MatchGap': np.array([5.0, 4.0]),
                'Month':  np.array([9.0, 9.0]),
                'MatchType': np.array([1.0, 1.0]),
                'HomeAway':  np.array([1.0, 0.0]),
                'Team':  np.array([60.0, 8.0]),
                'TeamRanking':  np.array([12.0, 7.0]),
                'PreScoreGoal':  np.array([12.0, 13.0]),
                'PrePreScoreGoal':  np.array([2.0, 0.0]),
                'PreGetGoal':  np.array([3.0, 1.0]),
                'PrePreGetGoal':  np.array([3.0, 3.0]),
                'RamosTime':  np.array([90.0, 68.0]),
                'RamosGoal':  np.array([0.0, 0.0]),
                'RamosAss':  np.array([0.0, 0.0]), }
    labels = np.array([0, 2])
    return features, labels


In [ ]:
def input_fn(features, labels, training=True, batch_size=256):
    """An input function for training or evaluating"""
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle and repeat if you are in training mode.
    if training:
        dataset = dataset.shuffle(1000).repeat()

    return dataset.batch(batch_size)


In [ ]:
# Feature columns describe how to use the input.
my_feature_columns = []
for key in train.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))


In [ ]:
# Build a DNN with 2 hidden layers with 30 and 10 hidden nodes each.
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    # Two hidden layers of 30 and 10 nodes respectively.
    hidden_units=[50, 10],
    # The model must choose between 3 classes.
    n_classes=3)


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpyx_rzbe1', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [ ]:
# Train the Model.
classifier.train(
    input_fn=lambda: input_fn(train, train_y, training=True),
    steps=20000)


Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpyx_rzbe1/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 9.353117, step = 0
INFO:tensorflow:global_step/sec: 240.44
INFO:tensorflow:loss = 1.6909605, step = 100 (0.417 sec)
INFO:tensorflow:global_step/sec: 270.732
INFO:tensorflow:loss = 1.2836117, step = 200 (0.372 sec)
INFO:tensorflow:global_step/sec: 

In [ ]:
# Generate predictions from the model
expected = ['Win']
predict_x = {
    'MatchGap': [13],
    'Month': [101],
    'MatchType': [1],
    'HomeAway': [1],
    'Team': [89],
    'TeamRanking': [2],
    'PreScoreGoal': [1],
    'PrePreScoreGoal': [3],
    'PreGetGoal': [4],
    'PrePreGetGoal': [2],
    'RamosTime': [90],
    'RamosGoal': [0],
    'RamosAss': [0],
}

def input_fn(features, batch_size=256):
    """An input function for prediction."""
    # Convert the inputs to a Dataset without labels.
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

predictions = classifier.predict(
    input_fn=lambda: input_fn(predict_x))



In [ ]:
for pred_dict, expec in zip(predictions, expected):
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]

    print('Prediction is "{}" ({:.1f}%), expected "{}"'.format(
        RESULTS[class_id], 100 * probability, expec))


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpyx_rzbe1/model.ckpt-20000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Prediction is "Lose" (99.6%), expected "Win"
